In [153]:
import numpy as np
from random import random

class Optimizador:
    def __init__(self, f, x0, max_iter = 10000, tol = 0.001):
        self.f = f
        self.x0 = x0
        self.max_iter = max_iter
        self.tol = tol
    
    # Función que calcula la gradiente de f en el punto xk mediante diferencias centrales
    def gradiente(self,xk, eps = 1e-6):
        size = len(xk)
        grad = np.zeros(size, dtype = float)
        for i in range(size):
            x_f = xk.copy(); 
            x_f[i] += eps
            x_b = xk.copy(); x_b[i] -= eps
            grad[i] = (self.f(x_f) - self.f(x_b)) / (2 * eps)
        return grad    

    # Función que calcula la hessiana de f en el punto xk mediante diferencias centrales
    def hessiana(self, xk, eps = 1e-3):
        size = len(xk)
        hess = np.zeros((size, size))
        for j in range(size):
            # Primera derivada
            dx_ff = xk.copy(); dx_ff[j] += eps;
            dx_bb = xk.copy(); dx_ff[j] += eps;
            grad_f = self.gradiente(dx_ff)
            grad_b = self.gradiente(dx_bb)
            for i in range(j+1):
                # Segunda derivada
                hess[i, j] = (grad_f[i] - grad_b[i]) / (2 * eps)
                hess[j, i] = (grad_f[i] - grad_b[i]) / (2 * eps)
        return hess
    
    # Función que aproxima el mínimo con el método de Newton (alpha estática)
    def Newton(self, xk, alpha, eps = 1e-6, max_iter = 1000): 
        for k in range(max_iter):
            grad = self.gradiente(xk)
            if(np.linalg.norm(grad) < eps): break
            hess = self.hessiana(xk)
            try: # Si la hessiana no es positiva definida causa error
                pk = np.linalg.solve(hess, -grad)
            except np.linalg.LinAlgError as err:
                print("La hessiana no es positiva definida")
                break
            xk = xk + alpha * pk
        return xk
    
    # Función que calcula el tamaño del paso, haciendo que se cumpla los criterios de Wolfe
    def BLS(self, xk, alpha_0, pk, c):
        alpha = alpha_0
        while self.f(xk + alpha * pk) > self.f(xk) + c * alpha * self.gradiente(xk) @ pk:
            p = random()
            alpha = p * alpha
        return alpha
        
    # Función que aproxima el mínimo con el método de Newton (alpha varía) pero no con modificación a la Hessiana    
    def LSN(self, xk):
        for k in range(self.max_iter):
            grad = self.gradiente(xk)
            if(np.linalg.norm(grad) < self.tol): break
            Bk = self.hessiana(xk)
            try:
                pk = np.linalg.solve(Bk, - grad)
            except np.linalg.LinAlgError as err:
                print("La hessiana no es positiva definida")
                break
            xk = xk + self.BLS(xk, 1, pk, 0.5) * pk
        return xk

    #Función que vuelve una matriz en positiva definida
    def CAMI(self,A):
        beta = 0.001
        n = len(A)
        if min(np.diag(A)) > 0:
            t = 0
        else:
            t = -min(np.diag(A)) + beta
        for k in range(self.max_iter):
            try:
                np.linalg.cholesky(A + t * np.identity(n))
            except np.linalg.LinAlgError as err: # Si no es pos. definida, actualizamos
                t = max(2 * t, beta)
            else:
                break
        return A + t * np.identity(n)
    
    # Función que aproxima el mínimo con el método de Newton (alpha varía) y con modificación a la Hessiana
    def LSNM(self, xk):
        for k in range(self.max_iter):
            grad = self.gradiente(xk)
            if(np.linalg.norm(grad) < self.tol): break
            Bk = self.hessiana(xk)
            try:
                np.linalg.cholesky(Bk)
            except np.linalg.LinAlgError as err:
                Bk = self.CAMI(Bk)   
            pk = np.linalg.solve(Bk, - grad)
            xk = xk + self.BLS(xk, 1, pk, 0.5) * pk
        return xk
    
    #Función que calcula el mínimo por Broyden, Fletcher, Goldfarb y Shanno.
    def BFGS(self, xk, Hk):
        I = np.identity(len(xk))
        for k in range(self.max_iter):
            grad = self.gradiente(xk)
            if(np.linalg.norm(grad) < self.tol): break
            #Dirección de Descenso
            pk = -Hk @ grad
            #Calculamos el tamaño del paso
            alpha = self.BLS(xk, 1, pk, 0.5)
            #Nueva x
            sk = alpha*pk; sk_t = sk.transpose()
            xk = xk + sk
            #Calculamos yk y phi_k
            yk = self.gradiente(xk) - grad; yk_t = yk.transpose();
            if(yk_t@sk == 0): break
            phi_k = 1.0/(yk_t@sk)
            #Calculamos la nueva hessiana inversa
            Hk = (I - phi_k*sk@yk_t)@Hk@(I - phi_k*sk@yk_t) + phi_k * sk @ sk_t
        return xk
    
    # Función que devuelve el óptimo calculado con el método elegido
    def optimiza(self, metodo):
        if metodo == "Newton":
            res = self.Newton(self.x0,0.1)
        elif metodo == "LSN":
            res = self.LSN(self.x0)
        elif metodo == "LSNM":
            res = self.LSNM(self.x0)
        elif metodo == "BFGS":
            res = self.BFGS(self.x0, np.identity(len(self.x0)))
        else:
            print("Método no encontrado")
        return res

In [154]:
def Rosenbrock(x, a = 1, b = 100):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

In [155]:
print("Prueba de aproximaciones")
opt = Optimizador(f = Rosenbrock,x0 = np.array([0.9, 1.1]))
print("Newton:",opt.optimiza("Newton"))
print("LSN:",opt.optimiza("LSN"))
print("LSNM", opt.optimiza("LSNM"))
print("BFGS", opt.optimiza("BFGS"))

Prueba de aproximaciones
Newton: [0.99999951 0.99999903]
LSN: [0.99954927 0.99909859]
LSNM [1.0004675  1.00093506]
BFGS [1.00083624 1.00167563]


#### Optimización Camara

In [160]:
import math
import pandas as pd

data=pd.read_csv('crime_data.csv')
#Filtramos por fecha
fecha_inicio = '2019-12-25'
fecha_fin = '2019-12-25'
data = data.loc[((data['date']>=fecha_inicio) & (data['date']<=fecha_fin))]
#Nos quedamos con sólo longitud y latitud
data = data[{'lat','long'}]
#Lo convertimos en un numpy array
crimes = data.to_numpy()

n = 8 # Número de cameras
cameras = crimes[:8].copy()

In [187]:
# Regresa la distancia entre dos ountos que se encuentran  en lat y lon
def haversine(p1, p2):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1[0]);p1_lon = math.radians(p1[1]);
    p2_lat = math.radians(p2[0]);p2_lon = math.radians(p2[1]);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    c = 2 * math.asin(math.sqrt(h)) 
    r = 6371 # Radio de la tierra en km
    return c * r

def distancia_min(x, crimenes = crimes):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(len(x)):
        for j in range(len(crimenes)):
            dist += haversine(x[i], crimenes[j])
        for j in range(len(x)):
            if(i != j):
                dist += 1/(haversine(x[i], x[j]))
    return dist 

distancia_min(cameras)

3637.3671451270766

In [182]:
#Pasamos de un arreglo de (n,2) -> (2n,1)
cam_2 = np.zeros(2*len(cameras))
j = 0
for i in range(len(cameras)):
    cam_2[j] = (cameras[i])[0]
    cam_2[j+1] = (cameras[i])[1]
    j = j+2
    
cri_2 = np.zeros(2*len(crimes))
j = 0
for i in range(len(crimes)):
    cri_2[j] = (crimes[i])[0]
    cri_2[j+1] = (crimes[i])[1]
    j=j+2
    
#Modificamos haversine y distancia
def haversine_mod(p1_a,p1_o, p2_a,p2_o):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1_a);p1_lon = math.radians(p1_o);
    p2_lat = math.radians(p2_a);p2_lon = math.radians(p2_o);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    if(h>1): h = 1
    c = 2 * math.asin(math.sqrt(abs(h))) 
    r = 6371 # Radio de la tierra en km
    return c * r

def distancia_min_mod(x, crimenes=cri_2):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(int(len(x)/2)):
        for j in range(int(len(crimenes)/2)):
            dist += haversine_mod(x[i],x[i+1], crimenes[j],crimenes[j+1])
        for j in range(int(len(x)/2)):
            if(i != j):
                hv = haversine_mod(x[i],x[i+1], x[j],x[j+1])
                if( hv != 0):
                    dist += 1/hv
    return dist 

distancia_min_mod(cam_2)

1820558.5003234304

In [189]:
print("Prueba de aproximación")
opt = Optimizador(f = distancia_min_mod,x0 = cam_2,max_iter = 1000)
print("LSNM", opt.optimiza("LSNM"))

Prueba de aproximación
LSNM [ 19.15488475 -99.14215626  19.26789592 -99.12737308  19.27825988
 -99.11657003  19.26073206 -99.14607932  20.24206625 -99.02906
  19.35711    -99.09761     19.31785    -99.06398     19.51575
 -99.13508   ]


In [192]:
xls = np.array([19.15488475,-99.14215626,19.26789592,-99.12737308,19.27825988,-99.11657003,19.26073206,-99.14607932,
                20.24206625,-99.02906,19.35711,-99.09761,19.31785,-99.06398,19.51575,-99.13508])
xlsnm = np.array([[19.15488475,-99.14215626],[19.26789592,-99.12737308],[19.27825988,-99.11657003],[19.26073206,-99.14607932],
                  [20.24206625,-99.02906],[19.35711,-99.09761],[19.31785,-99.06398],[19.51575,-99.13508]])
print("Distancias")
print("Modificada", distancia_min_mod(xls))
print("Original",distancia_min(xlsn))

Distancias
Modificada 1819624.373671123
Original 3637.366946362842
